In [1]:
from qiskit import *
from qiskit.circuit.library import QFT
from qiskit.circuit.random import random_circuit
from qiskit import qasm3
import networkx as nx
import numpy as np
from scipy import sparse
import time
from os import getcwd

from operations_graph import OperationsGraph
from route_forcing import RouteForcing

In [2]:
def decompose_to_cx(qc):
    count = 0
    while qc.num_nonlocal_gates() != len(qc.get_instructions('cx')) and count < 20000000:
        qc = qc.decompose()
        count += 1
        if count == 20000000:
            print('Decomposition stopped by count!', flush=True)
            count = 0
    
    return qc

In [3]:
def get_depth(qubits, mapper):
    circ = QuantumCircuit(qubits, qubits)

    for op in mapper.code:
        grid = int(np.sqrt(qubits))
        if op == 'NEXT':
            continue
        if len(op) == 2:
            circ.h(grid*op[1][0] + op[1][1])
        elif op[1] == op[2]:
            circ.h(grid*op[1][0] + op[1][1])
        else:
            circ.swap(grid*op[1][0] + op[1][1], grid*op[2][0] + op[2][1])

    return circ.depth()

In [4]:
circ = QFT(100)
circ = circ.decompose()
print()

for i,gate in enumerate(circ.data):
    print(gate.operation.name)

h
cp
h
cp
cp
h
cp
cp
cp
h
cp
cp
cp
cp
h
cp
cp
cp
cp
cp
h
cp
cp
cp
cp
cp
cp
h
cp
cp
cp
cp
cp
cp
cp
h
cp
cp
cp
cp
cp
cp
cp
cp
h
cp
cp
cp
cp
cp
cp
cp
cp
cp
h
swap
swap
swap
swap
swap


In [5]:
qubits_arr = []
for i in range(1, 11):
    qubits_arr.append((i*10)**2)


In [6]:
filename = "res.csv"
qubits = 100

G = nx.grid_2d_graph(int(np.sqrt(qubits)), int(np.sqrt(qubits)))
A = nx.adjacency_matrix(G).toarray()

adj_list = []
for i in range(len(A)):
    for j in range(len(A[0])):
        if A[i][j] == 1:
            adj_list.append([i, j])

pos = {n:n for n in G.nodes()}
edge_fidelity = {e:1.0 for e in G.edges()} # edge fidelity of 1 in all edges
edge_error_rate = {e:1-edge_fidelity[e] for e in G.edges()}

nx.set_node_attributes(G, pos, 'pos')

# Hyperparameters of the Route-Forcing algorithm
p = 0.2
k = 4

# QFT

circ = qasm3.load("../../input/qft_qiskit100.qasm")
# circ = decompose_to_cx(circ)
circ = circ.decompose()

op_graph = OperationsGraph(qiskit_circuit=circ)
# future_interactions = op_graph.get_future_interactions()
# print(future_interactions)
# for int_lev in future_interactions:
#     for op_ind in int_lev:
#         print(op_graph.operations[op_ind])
# exit(0)

start_time = time.time()
mapper = RouteForcing(op_graph=op_graph, topology=G)
initial_placement = mapper.place('trivial')

mapper.map(k=k, initial_placement=initial_placement, swap_penalization=-p, conv_steps=1000000)

end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

print(f"Time: {elapsed_time}s")
print(f"qubits:{qubits}")
print('SWAP gates:', mapper.added_swaps)
print('Resulting depth:', get_depth(qubits, mapper), "\n") # Obtained from Qiskit

with open("res.csv", "a") as f:
    f.write(f"{qubits},")
    f.write(f"{elapsed_time},")
    f.write(f"{mapper.added_swaps};")

Time: 0.686s
qubits:100
SWAP gates: 11942
Resulting depth: 872 



In [7]:
filename = "res.csv"

with open("res.csv", "w") as f:
    f.write(f"qubits,")
    f.write(f"elapsed_time,")
    f.write(f"added_swaps;")

for qubits in qubits_arr:
    G = nx.grid_2d_graph(int(np.sqrt(qubits)), int(np.sqrt(qubits)))
    A = nx.adjacency_matrix(G).toarray()

    adj_list = []
    for i in range(len(A)):
        for j in range(len(A[0])):
            if A[i][j] == 1:
                adj_list.append([i, j])

    pos = {n:n for n in G.nodes()}
    edge_fidelity = {e:1.0 for e in G.edges()} # edge fidelity of 1 in all edges
    edge_error_rate = {e:1-edge_fidelity[e] for e in G.edges()}

    nx.set_node_attributes(G, pos, 'pos')

    # Hyperparameters of the Route-Forcing algorithm
    p = 0.2
    k = 4

    # QFT

    circ = QFT(qubits)
    circ = decompose_to_cx(circ)

    op_graph = OperationsGraph(qiskit_circuit=circ)

    start_time = time.time()
    mapper = RouteForcing(op_graph=op_graph, topology=G)
    initial_placement = mapper.place('trivial')

    mapper.map(k=k, initial_placement=initial_placement, swap_penalization=-p, conv_steps=1000000)

    end_time = time.time()
    elapsed_time = round(end_time - start_time, 3)

    print(f"Time: {elapsed_time}s")
    print(f"qubits:{qubits}")
    print('SWAP gates:', mapper.added_swaps)
    print('Resulting depth:', get_depth(qubits, mapper), "\n") # Obtained from Qiskit

    with open("res.csv", "a") as f:
        f.write(f"{qubits},")
        f.write(f"{elapsed_time},")
        f.write(f"{mapper.added_swaps};")

Time: 1.145s
qubits:100
SWAP gates: 18356
Resulting depth: 1762 

Time: 23.57s
qubits:400
SWAP gates: 332593
Resulting depth: 8082 

Time: 398.094s
qubits:900
SWAP gates: 1807640
Resulting depth: 20551 



/home/alejandro/projects/cquant/.venv/lib/python3.11/site-packages/qiskit/circuit/library/blueprintcircuit.py:63: RuntimeWarning: precision loss in QFT. The rotation needed to represent 1599 entanglements is smaller than the smallest normal floating-point number.
  self._check_configuration()


In [ ]:
G = nx.grid_2d_graph(int(np.sqrt(qubits)), int(np.sqrt(qubits)))
A = nx.adjacency_matrix(G).toarray()

adj_list = []
for i in range(len(A)):
    for j in range(len(A[0])):
        if A[i][j] == 1:
            adj_list.append([i, j])

pos = {n:n for n in G.nodes()}
edge_fidelity = {e:1.0 for e in G.edges()} # edge fidelity of 1 in all edges
edge_error_rate = {e:1-edge_fidelity[e] for e in G.edges()}

nx.set_node_attributes(G, pos, 'pos')

In [ ]:
# Hyperparameters of the Route-Forcing algorithm
p = 0.2
k = 4

In [ ]:
# Random Circuit -- 0.3s

depth = 40
circ = random_circuit(qubits, depth, max_operands=2)

op_graph = OperationsGraph(qiskit_circuit=circ)

mapper = RouteForcing(op_graph=op_graph, topology=G)
initial_placement = mapper.place('trivial')

mapper.map(k=k, initial_placement=initial_placement, swap_penalization=-p, conv_steps=10000)

print('SWAP gates:', mapper.added_swaps)
print('Resulting depth:', get_depth(qubits, mapper)) # Obtained from Qiskit

In [ ]:
# QFT -- 4.3s

circ = QFT(qubits)
circ = decompose_to_cx(circ)

op_graph = OperationsGraph(qiskit_circuit=circ)

mapper = RouteForcing(op_graph=op_graph, topology=G)
initial_placement = mapper.place('trivial')

mapper.map(k=k, initial_placement=initial_placement, swap_penalization=-p, conv_steps=10000)

print('SWAP gates:', mapper.added_swaps)
print('Resulting depth:', get_depth(qubits, mapper)) # Obtained from Qiskit